In [17]:
########################################################################################################################
# This script organizes the modeling performance results across point prediction models into a single spreadsheet
########################################################################################################################

In [1]:
########################################################################################################################
# Import packages
########################################################################################################################
import numpy as np
import os
import pandas as pd
import warnings
from _Helper_Scripts.binary_metrics import binary_metrics, flagged_at_top_k_ppv,  nb_weight_from_pt, threshold_at_specificity_k 
from itertools import product
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [2]:
########################################################################################################################
# USER_SPECIFIC SETTING
# C_LIST: A list of different numbers of feature encounteres to be included
# D_LIST: A list of different maximum widths of the look-back window in days
# ABLATION: Boolean. False for pre-ablation modeling and True for post-ablation modeling
# IMPUTE_LIST: A list of strings representing the imputation methods adopted
# CSL: Boolean. False for standard learning and True for cost-sensitive learning
# DECIMALS: A integer representing the number of decimal points to display for performance statistics
# PT: A float representing the policy decision threshold for calculating once-off (standardized) net benefit
# IN_DIR: The input path of the directories storing the modeling results 
# (e.g., the directory with the subdirectory RiskPath_Results/) 
# OUT_DIR: The output path of the directory storing the overall performance statistics
########################################################################################################################
C_LIST: list[int] = [2, 3, 4]
D_LIST: list[int] = [60, 120, 180]
ABLATION: bool = False
IMPUTE_LIST: list[str] = ['Zero', 'Mean', 'Median']
CSL: bool = False
DECIMALS: int = 2
PT: float = 1 / 11
IN_DIR: str = ''
OUT_DIR: str = '_Final_Results/'

In [ ]:
############################################################################################################
# Extract probability estimates
############################################################################################################
files_needed: dict[tuple[int, int, str], str] = {}
base_dir: str = 'RiskPath_Results_Ablated' if ABLATION else 'RiskPath_Results/'
base_dir = os.path.join(IN_DIR, base_dir, 'Predicted_Probabilities/')
for C, D in product(C_LIST, D_LIST):
    for impute in IMPUTE_LIST:
        file_path: str = f'{base_dir}{C}_encounters_{D}_days_{impute}'
        if ABLATION:
            file_path += '_Ablated'
        file_path += '.csv'
        if CSL:
            file_path = file_path.replace('.csv', '_CSL.csv')
        files_needed[(C, D, impute)] = file_path
for k, v in files_needed.items():
    assert os.path.exists(v)
    # print(k, v)

In [15]:
########################################################################################################################
# Specify the output path of the CSV file
########################################################################################################################
os.makedirs(OUT_DIR, exist_ok=True)
suffix_1: str = '_POST' if ABLATION else '_PRE'
suffix_2: str = '_CSL' if CSL else ''
suffix: str = suffix_1 + suffix_2
out_file_name: str = os.path.join(OUT_DIR, f'Longitudinal_Performance_Imputation_Sensitivity{suffix}.csv')

In [17]:
########################################################################################################################
# Create a dictionary to store output pandas.DataFrames
########################################################################################################################
out_dict_list: list[dict] = []

In [ ]:
############################################################################################################
# Load the probability estimates files
############################################################################################################
for dict_key, file_path in files_needed.items():
    dict_key_str: str = f'{dict_key[0]}_{dict_key[1]}_{dict_key[2]}'
    df_cur: pd.DataFrame = pd.read_csv(file_path)
    
    ########################################################################################################
    # Extract the true labels and estimated probabilities
    ########################################################################################################
    y_test = df_cur['y_test'].to_numpy()
    y_prob = df_cur['y_prob'].to_numpy()

    ########################################################################################################
    # Define the selection cutoff for capacity-aligned results
    ########################################################################################################
    flag_1 = flagged_at_top_k_ppv(y_prob, k=1)
    flag_2 = flagged_at_top_k_ppv(y_prob, k=2)
    flag_5 = flagged_at_top_k_ppv(y_prob, k=5)
    spec_99 = threshold_at_specificity_k(y_test, y_prob, k=99)
    spec_95 = threshold_at_specificity_k(y_test, y_prob, k=95)
    spec_90 = threshold_at_specificity_k(y_test, y_prob, k=90)
    
    ########################################################################################################
    # Compute the net benefit weight
    ########################################################################################################
    nbw = nb_weight_from_pt(PT)    # nbw = p_t / (1 - p_t)
    
    ########################################################################################################
    # Obtain the lists of needed statistics
    ########################################################################################################
    default_stat = binary_metrics(y_true=y_test, y_prob=y_prob, y_pred_override=None, threshold=0.5, nb_weight=nbw, decimals=DECIMALS)
    top_1_stat = binary_metrics(y_true=y_test, y_prob=y_prob, y_pred_override=flag_1, threshold=0.5, nb_weight=nbw, decimals=DECIMALS)
    top_2_stat = binary_metrics(y_true=y_test, y_prob=y_prob, y_pred_override=flag_2, threshold=0.5, nb_weight=nbw, decimals=DECIMALS)
    top_5_stat = binary_metrics(y_true=y_test, y_prob=y_prob, y_pred_override=flag_5, threshold=0.5, nb_weight=nbw, decimals=DECIMALS)
    spec_99_stat = binary_metrics(y_true=y_test, y_prob=y_prob, y_pred_override=None, threshold=spec_99, nb_weight=nbw, decimals=DECIMALS)
    spec_95_stat = binary_metrics(y_true=y_test, y_prob=y_prob, y_pred_override=None, threshold=spec_95, nb_weight=nbw, decimals=DECIMALS)
    spec_90_stat = binary_metrics(y_true=y_test, y_prob=y_prob, y_pred_override=None, threshold=spec_90, nb_weight=nbw, decimals=DECIMALS)

    ########################################################################################################
    # Create a dictionary for the current set of performance statistics
    ########################################################################################################
    results_cur: dict[str, float] = {}
    
    ########################################################################################################
    # Organize the statistics
    ########################################################################################################
    stat_dict: dict[str, list] = {'': default_stat,
                                  '@top1%': top_1_stat,
                                  '@top2%': top_2_stat,
                                  '@top5%': top_5_stat,
                                  '@99Spec': spec_99_stat,
                                  '@95Spec': spec_95_stat,
                                  '@90Spec': spec_90_stat}
    for k, v in stat_dict.items():
        results_cur |= {f'{i}{k}': j for i, j in v.items() if not i.endswith('_LIST')}
    results_cur = {k: results_cur[k] for k in sorted(results_cur.keys())}
    results_cur = {'Setting': dict_key_str} | results_cur
    out_dict_list.append(results_cur)

In [23]:
########################################################################################################################
# Export out_dict_list as a CSV file
########################################################################################################################
df_out: pd.DataFrame = pd.DataFrame.from_records(out_dict_list)
df_out = df_out.set_index('Setting').T
df_out.to_csv(out_file_name, index=True)